In [60]:
def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield json.loads(l)

In [61]:
import pandas as pd
import gzip
import json

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('data/Video_Games_5.json.gz')

In [62]:
df.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,5.0,True,"10 17, 2015",A1HP7NVNPFMA4N,0700026657,Ambrosia075,"This game is a bit hard to get the hang of, bu...",but when you do it's great.,1445040000,NaN,NaN,NaN
1,4.0,False,"07 27, 2015",A1JGAP0185YJI6,0700026657,travis,I played it a while but it was alright. The st...,"But in spite of that it was fun, I liked it",1437955200,NaN,NaN,NaN
2,3.0,True,"02 23, 2015",A1YJWEXHQBWK2B,0700026657,Vincent G. Mezera,ok game.,Three Stars,1424649600,NaN,NaN,NaN
3,2.0,True,"02 20, 2015",A2204E1TH211HT,0700026657,Grandma KR,"found the game a bit too complicated, not what...",Two Stars,1424390400,NaN,NaN,NaN
4,5.0,True,"12 25, 2014",A2RF5B5H74JLPE,0700026657,jon,"great game, I love it and have played it since...",love this game,1419465600,NaN,NaN,NaN


In [63]:
df.overall.value_counts()

overall
5.0    299759
4.0     93654
3.0     49146
1.0     30883
2.0     24135
Name: count, dtype: int64

In [64]:
df['text'] = df['summary'] + ' ' + df['reviewText']
df = df[['overall', 'text']]
df.head()

,overall,text
0,5.0,but when you do it's great. This game is a bit...
1,4.0,"But in spite of that it was fun, I liked it I ..."
2,3.0,Three Stars ok game.
3,2.0,Two Stars found the game a bit too complicated...
4,5.0,"love this game great game, I love it and have ..."


In [65]:
import re

def clean_text(text):
    text = str(text)
    # text = text.lower()
    # text = re.sub(r'http[s]?://\S+', '', text)
    # text = re.sub(r'\[.*?\]\(.*?\)', '', text)
    # text = re.sub(r'@\w+', '', text)
    text = re.sub(r"([.,!?])", r" \1 ", text)  # E.g., convert "end." to "end . " ; \1 indicates a matched character
    text = re.sub(r'[^a-zA-Z\s\']', '', text)
    text = re.sub(r'\n', ' ', text)
    # text = re.sub(r's+', ' ', text)
    return text.strip()
    
# Apply text cleaning to review titles and content
df['text'] = df.text.apply(clean_text)
df.head()

,overall,text
0,5.0,but when you do it's great This game is a bi...
1,4.0,But in spite of that it was fun I liked it I...
2,3.0,Three Stars ok game
3,2.0,Two Stars found the game a bit too complicated...
4,5.0,love this game great game I love it and have...


In [66]:
df['label'] = pd.cut(df['overall'],bins=[0,3,5],labels=['negative','positive'], right=True)
df['text_length'] = df['text'].apply(lambda x: len(x))
df.label.value_counts()

label
positive    393413
negative    104164
Name: count, dtype: int64

In [ ]:
df = df[df['text_length'] > 100]
df.head()

,overall,text,label,text_length
1,4.0,But in spite of that it was fun I liked it I...,positive,342
5,4.0,Anno i liked a lot some time that i haven't p...,positive,110
6,1.0,Avoid This Game Filled with Bugs I'm an avid ...,negative,507
7,5.0,A very good game balance of skill with depth o...,positive,770
8,5.0,Anno more like anno I have played the old an...,positive,903


In [68]:
import collections
by_label = collections.defaultdict(list)
for _,row in df.iterrows():
    by_label[row.label].append(row.to_dict())

by_label.keys()

dict_keys(['positive', 'negative'])

In [69]:
import numpy as np
final_list = []
np.random.seed(1234)

for _, item_list in sorted(by_label.items()):

    np.random.shuffle(item_list)
    
    n_total = len(item_list)
    n_sample = int(0.1 * n_total)
    
    # Give data point a split attribute
    for item in item_list[:n_sample]:
        item['split'] = 'T'
    
    for item in item_list[n_sample:]:
        item['split'] = 'F'

    # Add to final list
    final_list.extend(item_list) 

In [70]:
final_df = pd.DataFrame(final_list)
final_df.split.value_counts()

split
F    317300
T     35254
Name: count, dtype: int64

In [71]:
df_sample = final_df[final_df['split']== 'T'][['label', 'text']]
df_positive = df_sample[(df_sample['label'] == 'positive')].iloc[:3000,:]
df_negative = df_sample[df_sample['label'] == 'negative'].iloc[:3000,:]
df_output = pd.concat([df_positive, df_negative])

In [72]:
df_output.head()

,label,text
89316,positive,Rise of Flight The First Great Air War Iron C...
89317,positive,Funny and Creative No tech issues for me Pla...
89318,positive,For the price was shocked how good this looks ...
89319,positive,A little slow at the start but still awesome ...
89320,positive,If the formula works why change it This ga...


In [73]:
df_output.label.value_counts()

label
positive    3000
negative    3000
Name: count, dtype: int64

In [74]:
df_output.to_csv('data/game_reviews.csv')